<a href="https://colab.research.google.com/github/komal-shahzada/Syntecxhub-Internship-tasks/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install gradio pandas scikit-learn -q

import pandas as pd
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Dataset Preparation
# Using a sample dataset for quick execution
data = {
    'movie_title': ['The Dark Knight', 'Inception', 'Interstellar', 'The Avengers', 'Iron Man', 'Joker', 'The Godfather', 'Pulp Fiction', 'Fight Club', 'Forrest Gump'],
    'genre': ['Action Crime Drama', 'Action Sci-Fi Adventure', 'Adventure Sci-Fi Drama', 'Action Adventure Sci-Fi', 'Action Adventure Sci-Fi', 'Crime Drama Thriller', 'Crime Drama', 'Crime Drama', 'Drama', 'Drama Romance'],
    'description': ['Batman fights the Joker in Gotham.', 'Dream within a dream heist.', 'Space exploration through a wormhole.', 'Superheroes team up to save Earth.', 'Tony Stark builds a high-tech suit.', 'A failed comedian turns to crime.', 'The aging patriarch of an organized crime dynasty.', 'Life of two mob hitmen.', 'An insomniac forms an underground fight club.', 'A man with low IQ achieves great things.']
}

df = pd.DataFrame(data)

# 2. Metadata Cleaning & Feature Engineering
# Combining Genre and Description to create a 'Tags' column for the recommender
df['tags'] = df['genre'] + " " + df['description']

# 3. Content-Based Filtering Logic
# Using TF-IDF Vectorizer to convert text into numerical vectors
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['tags'])

# Calculating Cosine Similarity between all movies
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get movie recommendations
def get_recommendations(movie_name):
    try:
        # Find index of the movie that matches the title
        idx = df[df['movie_title'] == movie_name].index[0]

        # Get pairwise similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort movies based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get indices of the 3 most similar movies (excluding itself)
        movie_indices = [i[0] for i in sim_scores[1:4]]

        # Return the titles of the top 3 recommendations
        return df['movie_title'].iloc[movie_indices].tolist()
    except:
        return ["Movie not found in the local database! Try 'Inception' or 'Joker'."]

# 4. Attractive Gradio Interface Design
with gr.Blocks(theme=gr.themes.Soft(primary_hue="purple", secondary_hue="indigo")) as movie_app:
    gr.Markdown("""
    # 🎬 Smart Movie Recommender
    ### Discover your next favorite movie based on AI-powered content similarity.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            input_movie = gr.Dropdown(
                choices=df['movie_title'].tolist(),
                label="Select a Movie you Liked",
                info="Our AI will find similar movies based on genre and story."
            )
            search_btn = gr.Button("Find Recommendations", variant="primary")

        with gr.Column(scale=1):
            output_list = gr.JSON(label="Recommended for You")

    # Linking the search button to the prediction function
    search_btn.click(fn=get_recommendations, inputs=input_movie, outputs=output_list)

    gr.Markdown("---")
    gr.Markdown("© 2026 | SyntecxHub Machine Learning Task")

# 5. Launch the App
movie_app.launch(share=True)

/tmp/ipython-input-2278137430.py:52: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(primary_hue="purple", secondary_hue="indigo")) as movie_app:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c563eb608d6e672e81.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
